In [2]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

In [3]:
DATABASE_TYPE="postgresql"
USERNAME="mutakabbir"
PASSWORD="lightning"
HOST="localhost"
PORT=5432
DATABASE_NAME="postgres"
FIRE_SUBDIVISION_TABLE = "fire_division_dataset"

In [4]:
engine = create_engine(f"{DATABASE_TYPE}://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}")

In [5]:
query_fire = """select f."YEAR", f."MONTH", f."DAY", f."REP_DATE", f."CALC_HA", f."CAUSE", f."geometry" from fire f"""
query_subdivision = """select * from subdivision_dataset sd"""

In [6]:
fire_gdf = gpd.read_postgis(
    sql=query_fire, 
    con=engine,
    geom_col="geometry",
    crs="EPSG:4326",)  

In [7]:
fire_gdf

,YEAR,MONTH,DAY,REP_DATE,CALC_HA,CAUSE,geometry
0,2006,4,9,2006-04-09,6.741532,H,"POLYGON ((-120.25300 55.87807, -120.25293 55.8..."
1,2006,4,15,2006-04-15,0.562148,H,"POLYGON ((-121.57551 55.60000, -121.57551 55.5..."
2,2006,5,29,2006-05-29,1.682437,H,"POLYGON ((-121.57539 55.46293, -121.57565 55.4..."
3,2006,5,18,2006-05-18,1.108936,H,"POLYGON ((-121.64146 55.70862, -121.64138 55.7..."
4,2006,4,24,2006-04-24,151.231090,H,"MULTIPOLYGON (((-120.47959 55.81054, -120.4791..."
...,...,...,...,...,...,...,...
59533,2006,7,17,2006-07-17,13.496292,L,"POLYGON ((-124.53606 55.46217, -124.53553 55.4..."
59534,2006,7,3,2006-07-03,181.530303,L,"MULTIPOLYGON (((-123.61535 55.55967, -123.6150..."
59535,2006,7,4,2006-07-04,2786.378984,L,"POLYGON ((-125.32666 56.08734, -125.32603 56.0..."
59536,2006,7,14,2006-07-14,5.241929,L,"POLYGON ((-125.27132 56.06404, -125.27152 56.0..."


In [8]:
division_gdf = gpd.read_postgis(
    sql=query_subdivision, 
    con=engine,
    geom_col="geometry",
    crs="EPSG:4326",)  

In [9]:
division_gdf

,cid,geometry
0,0,"MULTIPOLYGON (((-139.53267 69.53197, -139.5336..."
1,1,"MULTIPOLYGON (((-59.17563 47.55863, -59.17579 ..."
2,2,"MULTIPOLYGON (((-112.70518 67.67319, -112.7050..."
3,3,"MULTIPOLYGON (((-82.52584 42.47577, -82.52580 ..."
4,4,"POLYGON ((-112.31319 55.39631, -112.31267 55.3..."
...,...,...
62,62,"POLYGON ((-121.20810 50.56893, -121.20824 50.5..."
63,63,"POLYGON ((-118.79457 54.58827, -118.74824 54.5..."
64,64,"MULTIPOLYGON (((-121.00702 49.65631, -121.0066..."
65,65,"MULTIPOLYGON (((-80.76446 45.82386, -80.76447 ..."


In [10]:
fire_in_subdivision_gdf = gpd.sjoin(
    fire_gdf,
    division_gdf,
    how="inner",
    predicate="within").drop("index_right", axis=1)

In [11]:
fire_in_subdivision_gdf = fire_in_subdivision_gdf.rename(
    {
        "YEAR": "year",
        "MONTH": "month",
        "DAY": "day",
        "REP_DATE": "start_date",
        "CALC_HA": "area_burnt_ha",
        "CAUSE": "cause",
        "cid": "division_id"
    },
    axis = 1
)

In [12]:
# group to indexes
fire_in_subdivision_gdf = fire_in_subdivision_gdf.groupby(["division_id","start_date","area_burnt_ha","cause",]).first().reset_index()

In [13]:
# drop duplicates
fire_in_subdivision_gdf.drop_duplicates(inplace=True)

In [14]:
fire_in_subdivision_gdf.to_postgis(name=FIRE_SUBDIVISION_TABLE, con=engine, if_exists='replace', index=False)

/data/mutakabbir/Lightning/venv/lib/python3.8/site-packages/geopandas/io/sql.py:389: UserWarning: Could not parse CRS from the GeoDataFrame. Inserting data without defined CRS.
  srid = _get_srid_from_crs(gdf)


In [15]:
with engine.connect() as con:
    con.execute(f'ALTER TABLE {FIRE_SUBDIVISION_TABLE} ADD PRIMARY KEY ("division_id","start_date","area_burnt_ha","cause");')

In [28]:
# # Operation too long
# fire_overlaps_subdivision_gdf = gpd.sjoin(
#     fire_not_in_sub_division,
#     division_gdf,
#     how="inner",
#     predicate="overlaps").drop("index_right", axis=1)

In [16]:
fire_in_subdivision_gdf

,division_id,start_date,area_burnt_ha,cause,year,month,day,geometry
0,0,1960-05-10,6422.863612,L,1960,5,10,"POLYGON ((-136.74856 66.77644, -136.74118 66.7..."
1,0,1960-06-12,679.814663,L,1960,6,12,"POLYGON ((-136.63157 65.80181, -136.62934 65.8..."
2,0,1960-07-15,329.731143,H,1960,7,15,"POLYGON ((-136.87935 67.32493, -136.87387 67.3..."
3,0,1960-07-20,559.296563,H,1960,7,20,"POLYGON ((-124.77179 60.06496, -124.77089 60.0..."
4,0,1960-08-03,724.944073,H,1960,8,3,"POLYGON ((-133.87588 60.12462, -133.87935 60.1..."
...,...,...,...,...,...,...,...,...
46478,66,2019-06-02,1.046225,H,2019,6,2,"POLYGON ((-114.95344 51.21220, -114.95340 51.2..."
46479,66,2020-09-04,2417.256996,H,2020,9,4,"MULTIPOLYGON (((-115.26752 51.29490, -115.2677..."
46480,66,2020-10-05,2.773375,H,2020,10,5,"POLYGON ((-114.90744 51.12062, -114.90742 51.1..."
46481,66,2020-10-11,18.277458,H,2020,10,11,"POLYGON ((-114.74050 51.20659, -114.74049 51.2..."
